In [ ]:
push!(LOAD_PATH, "../src/")

using DataFrames
using Molecules
using Unitful
using Netsuriki
using CSV

In [ ]:
CH2_geom = """
C                  0.00000000    0.00000000    0.10308800
H                  0.00000000    0.99645900   -0.30926300
H                  0.00000000   -0.99645900   -0.30926300
"""
CH2_mol = Molecule(CH2_geom, 0, 3)
CH2_ν̃s = [1058.1217, 3125.2710, 3362.8179] .* 1.0u"1/cm";

In [ ]:
# Electronic

q_el = Electronic.q(CH2_mol)
Am_el = Electronic.Am(q_el)
Gm_el = Electronic.Gm(q_el)
Um_el = Electronic.Um()
Hm_el = Electronic.Hm()
Sm_el = Electronic.Sm(q_el)
CVm_el = Electronic.CVm()
Cpm_el = Electronic.Cpm();

In [ ]:
# Translational

q_tr = Translational.q(CH2_mol)
Am_tr = Translational.Am(q_tr)
Gm_tr = Translational.Gm(q_tr)
Um_tr = Translational.Um()
Hm_tr = Translational.Hm()
Sm_tr = Translational.Sm(q_tr)
CVm_tr = Translational.CVm()
Cpm_tr = Translational.Cpm();

In [ ]:
# Rotational

q_rot = Rotational.q(CH2_mol)
Am_rot = Rotational.Am(q_rot)
Gm_rot = Rotational.Gm(q_rot)
Um_rot = Rotational.Um()
Hm_rot = Rotational.Hm()
Sm_rot = Rotational.Sm(q_rot)
CVm_rot = Rotational.CVm()
Cpm_rot = Rotational.Cpm();

In [ ]:
# Vibrational

q_vib = Vibrational.q(CH2_ν̃s)
Am_vib = Vibrational.Am(q_vib)
Gm_vib = Vibrational.Gm(q_vib)
Um_vib = Vibrational.Um(CH2_ν̃s)
Hm_vib = Vibrational.Hm(CH2_ν̃s)
Sm_vib = Vibrational.Sm(CH2_ν̃s)
CVm_vib = Vibrational.CVm(CH2_ν̃s)
Cpm_vib = Vibrational.Cpm(CH2_ν̃s);

In [ ]:
# Total

q_tot = q_el * q_tr * q_rot * q_vib
Am_tot = Am_el + Am_tr + Am_rot + Am_vib
Gm_tot = Gm_el + Gm_tr + Gm_rot + Gm_vib
Um_tot = Um_el + Um_tr + Um_rot + Um_vib
Hm_tot = Hm_el + Hm_tr + Hm_rot + Hm_vib
Sm_tot = Sm_el + Sm_tr + Sm_rot + Sm_vib
CVm_tot = CVm_el + CVm_tr + CVm_rot + CVm_vib
Cpm_tot = Cpm_el + Cpm_tr + Cpm_rot + Cpm_vib;

In [ ]:
function thermo_J()
    DataFrame(
        Item = ["Total", "Electronic", "Translational", "Rotational", "Vibrational"],
        Am = [Am_tot, Am_el, Am_tr, Am_rot, Am_vib],
        Gm = [Gm_tot, Gm_el, Gm_tr, Gm_rot, Gm_vib],
        Um = [Um_tot, Um_el, Um_tr, Um_rot, Um_vib],
        Hm = [Hm_tot, Hm_el, Hm_tr, Hm_rot, Hm_vib],
        Sm = [Sm_tot, Sm_el, Sm_tr, Sm_rot, Sm_vib],
        CVm = [CVm_tot, CVm_el, CVm_tr, CVm_rot, CVm_vib],
        Cpm = [Cpm_tot, Cpm_el, Cpm_tr, Cpm_rot, Cpm_vib]
    )
end

CSV.write("CH2-thermo-J.csv", thermo_J())
@show thermo_J();

In [ ]:
function thermo_cal()
    df = thermo_J()
    map([:Am, :Gm, :Um, :Hm]) do x
        select!(df, :, x => (x -> x .|> u"kcal/mol") => x)
    end
    map([:Sm, :CVm, :Cpm]) do x
        select!(df, :, x => (x -> x .|> u"cal/mol/K") => x)
    end
    df
end

CSV.write("CH2-thermo-cal.csv", thermo_cal())
@show thermo_cal();

In [ ]:
@show thermo_cal()[:, [:Um, :CVm, :Sm]];

In [ ]:
function partation_functions()
    DataFrame(
        Item = ["Total (V=0)", "Vib (V=0)", "Electronic", "Translational", "Rotational"],
        Q = [q_tot / N_A, q_vib, q_el, q_tr / N_A, q_rot]
    )
end

CSV.write("CH2-partation-functions.csv", partation_functions())
@show partation_functions();